# STM8L-Discovery DMA example

Load [REMCU](https://remotemcu.com) shared libray

In [ ]:
.L libremcu.so

Add path with header files

In [ ]:
.I remcu_include

Including necessary header files. The [“remcu.h”](remcu_include/remcu.h) header must be always included before any MCU header files.

In [ ]:
#include "remcu.h"
#include "stm8l15x.h"

Connection to a debug server. The [**REMCU**](https://remotemcu.com/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/).
There is using the [remcu_connect2OpenOCD](https://remotemcu.com/api-v1-0#remcu_getVersion) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)

Before that, plug STM8L-Discovery board with debugger to PC

run openocd utility in command line:
```
./openocd -f interface/stlink-v2.cfg -f target/stm8l.cfg
```
Result:
![Image](img/openocd_unix.png)

In [ ]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const int timeout_sec = 3; // It can not be negative

In [ ]:
remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you want to use the GDB server of OpenOCD, uncomment and run code below:

In [ ]:
//const uint16_t default_openocd_gdb_port = 3333;
//remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port, timeout_sec)

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page) 
    
If connection were success... Jupyter Output:
![Image](img/success_connection_for_unix.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)):

In [ ]:
remcu_resetRemoteUnit(__HALT)

## ADC-DMA example
This snippet captures ADC data on PC7 pin of STM8L-Discovery and store to MCU's memory(RAM: at 0x0 address)

In [ ]:
  /*High speed external clock prescaler: 1*/
  CLK_SYSCLKDivConfig(CLK_SYSCLKDiv_1);

  /* Enable ADC1 clock */
  CLK_PeripheralClockConfig(CLK_Peripheral_ADC1, ENABLE);

  /* Enable DMA1 clock */
  CLK_PeripheralClockConfig(CLK_Peripheral_DMA1, ENABLE);

  /* Enable TIM1 clock */
  CLK_PeripheralClockConfig(CLK_Peripheral_TIM1, ENABLE);

In [ ]:
ADC_SamplingTime_TypeDef SamplingTime;

In [ ]:
  SamplingTime = ADC_SamplingTime_4Cycles;

  /* Initialize and configure ADC1 */
  ADC_Init(ADC1, ADC_ConversionMode_Single, ADC_Resolution_12Bit, ADC_Prescaler_1);
  ADC_SamplingTimeConfig(ADC1, ADC_Group_SlowChannels, SamplingTime);

  /* Enable ADC1 */
  ADC_Cmd(ADC1, ENABLE);

  /* Enable ADC1 Channels 3 - PC7 pin */
  ADC_ChannelCmd(ADC1, ADC_Channel_3, ENABLE); /* connected to Potentiometer RV */

In [ ]:
const uint16_t ADC_BUFFER_ADDRESS = 0x0;
const uint16_t ADC1_DR_ADDRESS  = 0x5344;
const uint16_t ADC1_8_DR_ADDRESS  = 0x5345;
const uint8_t ADC_BUFFER_SIZE = 0xFF;

In [ ]:
 /* Connect ADC to DMA channel 0 */
  SYSCFG_REMAPDMAChannelConfig(REMAP_DMA1Channel_ADC1ToChannel0);
  DMA_GlobalDeInit();
  DMA_DeInit(DMA1_Channel0);
  DMA_Init(DMA1_Channel0, ADC_BUFFER_ADDRESS,
           ADC1_DR_ADDRESS,           //set ADC1_8_DR_ADDRESS, if you want to use 8-bit mode of ADC
           ADC_BUFFER_SIZE,
           DMA_DIR_PeripheralToMemory,
           DMA_Mode_Circular,
           DMA_MemoryIncMode_Inc,
           DMA_Priority_High,
           DMA_MemoryDataSize_HalfWord           //set DMA_MemoryDataSize_Byte, if you want to use 8-bit mode of ADC
           );

  /* DMA Channel0 enable */
  DMA_Cmd(DMA1_Channel0, ENABLE);

  /* DMA enable */
  DMA_GlobalCmd(ENABLE);

  /* Enable ADC1 DMA requests*/
  ADC_DMACmd(ADC1, ENABLE);

  /* Start ADC1 Conversion using TIM1 TRGO*/
  ADC_ExternalTrigConfig(ADC1, ADC_ExtEventSelection_Trigger2, ADC_ExtTRGSensitivity_Rising);

In [ ]:
uint16_t TIM1_Prescaler;
uint16_t TIM1_Period;
uint8_t TIM1_RepetitionCounter;

In [ ]:
  TIM1_Prescaler = 0x0; 
  TIM1_Period = 0x2;
  TIM1_RepetitionCounter = 0;
  TIM1_TimeBaseInit(TIM1_Prescaler, TIM1_CounterMode_Up, TIM1_Period, TIM1_RepetitionCounter);
/* Master Mode selection: Update event */
  TIM1_SelectOutputTrigger(TIM1_TRGOSource_Update);
/* Enable TIM1 */
  TIM1_Cmd(ENABLE);

In [ ]:
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

xpl::figure fig;
xpl::linear_scale sx, sy;
xpl::lines line(sx, sy);
std::vector<uint16_t> adc_data(ADC_BUFFER_SIZE );
std::vector<int> x_line(ADC_BUFFER_SIZE );
int osc_data[ADC_BUFFER_SIZE] = {0};
size_t shift = 0;
int i = 0;
for( auto & x : x_line){x = i++;}
line.x = x_line;
line.y = adc_data;
fig.add_mark(line);
    
xpl::axis hx(sx), hy(sy);
hy.orientation = "vertical";
fig.add_axis(hx);
fig.add_axis(hy);

Connect a signal generator to PC7 pin of STM8L-Discovery board.
Plotting raw data of chip's memory:
>Note that Community and Education versions of the REMCU library have a 32-bytes limit on a memory operation. Therefore, if you need to copy big data, you can do it in parts.
Also the Community and Education versions have restrictions on memory region where you store and load data. You can use first 1 KBytes of first memory bank(STM8L chip). It is from 0x0 to 1024 address. To clarify that, see the Download page of your chip or the START_AVAILABLE_MEMORY_REGION and EMD_AVAILABLE_MEMORY_REGION constants in device_defines.h(is in remcu_include folder)

In [ ]:
ADC_Cmd(ADC1, DISABLE);
adc_data.back() = 0;
for(int i = 0; i < ADC_BUFFER_SIZE*sizeof(adc_data.front()); i += 0x20){
    remcu_loadFrMem(i, 32, (uint8_t*)(&adc_data.front()) + i);
}
shift =  DMA_GetCurrDataCounter(DMA1_Channel0);
ADC_Cmd(ADC1, ENABLE);

line.y = adc_data; //plot graph
fig

<details>
  <summary>You can get the strange signal <b>(click to show)</b></summary>

![Image](img/sunus_big_endian.png)
</details>
As STM8L is big endian processor, a byte reorder is needed:

In [ ]:
#include <netinet/in.h>

for(int i = 0; i < 0xFF; i++){
    uint16_t temp = adc_data[i];
    temp = htons(temp);
    adc_data[i] = temp;
}
line.y = adc_data; //plot graph
fig

<details>
  <summary>Signal get rigth wave <b>(click to show)</b></summary>

![Image](img/sinus_little_endian.png)
</details>
<details>
  <summary>But it has the artifact because the the ADC’s DMA channel saves its data in a circular fashion and keeps on overwriting old data until is suspended <b>(click to show)</b></summary>

![Image](img/artifact.png)
</details>

<details>
  <summary>Left shift of ADC data for a picture like an oscilloscope screen<b>(click to show)</b></summary>

![Image](img/sinus_little_endian_shift.png)
</details>


In [ ]:
for(int i = 0; i < ADC_BUFFER_SIZE; i++){ osc_data[i] = adc_data[i];}

shift =  ADC_BUFFER_SIZE - shift;

for(size_t i = 0; i < ADC_BUFFER_SIZE; i++){
        int shift_pos = (i + shift) % ADC_BUFFER_SIZE;
        adc_data[i] = osc_data[shift_pos];
    }
line.y = adc_data; //plot graph
fig

<details>
  <summary>Sinus wave like on oscilloscope screen:<b>(click to show)</b></summary>

![Image](img/after_shift.png)
</details>

## DAC-DMA example
This snippet generates a sine siganl on PF0 pin using the DAC and DMA modules of STM8L-Discovery. It can work with the previous snippet ([**ADC-DMA example**](#ADC-DMA-example)) simultaneously

In [ ]:
 const uint16_t MEM_ADDRESS = ADC_BUFFER_SIZE*sizeof(adc_data.front()) + 1;
 const uint8_t MEM_SIZE = 130;
 uint8_t SINUS_TABLE[130] = {110,115,121,126,131,137,142,147,
152,157,161,166,171,175,179,183,187,191,195,198,201,204,207,209,
211,213,215,216,218,219,219,220,220,220,220,219,218,217,216,214,
212,210,208,205,202,199,196,193,189,185,181,177,173,168,164,159,
154,149,144,139,134,129,123,118,113,107,102,97,91,86,81,76,
71,66,61,56,52,47,43,39,35,31,27,24,21,18,15,12,
10,8,6,4,3,2,1,0,0,0,0,1,1,2,4,5,7,9,11,13,16,19,22,25,29,33,37,41,45,49,54,59,
63,68,73,78,83,89,94,99,105,110, };

Load the sinus table into MCU's memory at MEM_ADDRESS address behind the ADC memory buffer:
>Note that Community and Education versions of the REMCU library have a 32-bytes limit on a memory operation. Therefore, if you need to copy big data, you can do it in parts.
Also the Community and Education versions have restrictions on memory region where you store and load data. You can use first 1 KBytes of first memory bank(STM8L chip). It is from 0x0 to 1024 address. To clarify that, see the Download page of your chip or the START_AVAILABLE_MEMORY_REGION and EMD_AVAILABLE_MEMORY_REGION constants in device_defines.h(is in remcu_include folder)

In [ ]:
for(int i = 0 ; i < MEM_SIZE*sizeof(SINUS_TABLE[0]); i+=10){
    remcu_store2mem(MEM_ADDRESS + i, (uint8_t*)SINUS_TABLE + i, 10);
}

Enable clock of DAC and TIMER 4 modules:

In [ ]:
  CLK_PeripheralClockConfig(CLK_Peripheral_DAC, ENABLE);
  CLK_PeripheralClockConfig(CLK_Peripheral_TIM4, ENABLE);

In [ ]:
  /* DMA channel3 Config -----------------------------------------------------*/
#define DAC_CH1RDHRH_ADDRESS      0x5388
#define DAC_CH1RD8_ADDRESS      0x5390
#define DAC_CH1RDHLH_ADDRESS      0x538C

  DMA_DeInit(DMA1_Channel3);
  DMA_Init(DMA1_Channel3,  MEM_ADDRESS,
           DAC_CH1RD8_ADDRESS,
           MEM_SIZE, DMA_DIR_MemoryToPeripheral, DMA_Mode_Circular,
           DMA_MemoryIncMode_Inc, DMA_Priority_High,
           DMA_MemoryDataSize_Byte
          );

  /* DMA1 Channel 3 enable */
  DMA_Cmd(DMA1_Channel3, ENABLE);
  DMA_GlobalCmd(ENABLE);

In [ ]:
  /* DAC Channel1 Config: 12bit right ----------------------------------------*/
  /* DAC deinitialize */
  DAC_DeInit();
  
  /* Fill DAC Init param DAC_Trigger_T4_TRGO and  DAC Channel1 Init */
  DAC_Init(DAC_Channel_1, DAC_Trigger_T4_TRGO, DAC_OutputBuffer_Enable);

  /* Enable DAC Channel1 */
  DAC_Cmd(DAC_Channel_1, ENABLE);
  
  /* Enable DMA for DAC Channel1 */
  DAC_DMACmd(DAC_Channel_1, ENABLE);

In [ ]:
  TIM4_DeInit();
  /* Time base configuration */
  TIM4_TimeBaseInit(TIM4_Prescaler_1, 0x1);
  
  /* TIM4 TRGO selection */
  TIM4_SelectOutputTrigger(TIM4_TRGOSource_Update);
  
  /* TIM4 enable counter */
  TIM4_Cmd(ENABLE);

To see DAC signal in the ADC viewer, link PC7 and PF0 pin of STM8L-Discovery and run code below:

In [ ]:
ADC_Cmd(ADC1, DISABLE);
adc_data.back() = 0;
for(int i = 0; i < ADC_BUFFER_SIZE*sizeof(adc_data.front()); i += 0x20){
    remcu_loadFrMem(i, 32, (uint8_t*)(&adc_data.front()) + i);
}
shift =  DMA_GetCurrDataCounter(DMA1_Channel0);
ADC_Cmd(ADC1, ENABLE);

for(int i = 0; i < 0xFF; i++){
    uint16_t temp = adc_data[i];
    temp = htons(temp);
    adc_data[i] = temp;
}

for(int i = 0; i < ADC_BUFFER_SIZE; i++){ osc_data[i] = adc_data[i];}

shift =  ADC_BUFFER_SIZE - shift;

for(size_t i = 0; i < ADC_BUFFER_SIZE; i++){
        int shift_pos = (i + shift) % ADC_BUFFER_SIZE;
        adc_data[i] = osc_data[shift_pos];
    }
line.y = adc_data; //plot graph
fig

<details>
  <summary>Result:<b>(click to show)</b></summary>

![](img/DAC_signal.png)
</details>

Disconnecting from the debug server. It is finalizing work with REMCU Library

In [ ]:
remcu_disconnect()